## Install Dependencies and Import Libraries

In [1]:
# Python version must be 3.6 ~ 3.10, which is officially supported for pyrealsense2 library
!pip install -r requirements.txt

In [2]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from ultralytics import YOLO
from datetime import datetime
import json
import shutil
import random
from pathlib import Path

## Dataset Preprocessing

In [4]:
import os
import json
import cv2
from pathlib import Path

# Config
dataset_root = Path("dataset")
splits = ["train", "val"]

# Folder setup
for split in splits:
    (dataset_root / split / "images").mkdir(parents=True, exist_ok=True)
    (dataset_root / split / "labels").mkdir(parents=True, exist_ok=True)

# Collect all class labels
all_labels = set()
for split in splits:
    for file in (dataset_root / "annotations" / split).glob("*.json"):
        with open(file) as f:
            data = json.load(f)
            all_labels.update(shape["label"] for shape in data["shapes"])
all_labels = sorted(all_labels)
class_map = {label: idx for idx, label in enumerate(all_labels)}

# Helper: Get bounding box from polygon
def polygon_to_bbox(points):
    xs = [pt[0] for pt in points]
    ys = [pt[1] for pt in points]
    x_min, x_max = min(xs), max(xs)
    y_min, y_max = min(ys), max(ys)
    return x_min, y_min, x_max, y_max

# Convert Labelme to YOLO bbox labels
def convert_to_yolo_bbox(json_file, split):
    with open(json_file) as f:
        data = json.load(f)

    image_name = json_file.stem + ".png"
    image_path = dataset_root / split / "images" / image_name
    original_image_path = dataset_root / "images" / split / data["imagePath"]
    image = cv2.imread(str(original_image_path))
    if image is None:
        raise FileNotFoundError(f"Image not found: {original_image_path}")
    h, w = image.shape[:2]

    yolo_labels = []

    for shape in data["shapes"]:
        label = shape["label"]
        cls_id = class_map[label]
        x_min, y_min, x_max, y_max = polygon_to_bbox(shape["points"])

        # Normalize bbox
        x_c = ((x_min + x_max) / 2) / w
        y_c = ((y_min + y_max) / 2) / h
        bw = (x_max - x_min) / w
        bh = (y_max - y_min) / h

        yolo_labels.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")

    # Save YOLO label file
    label_path = dataset_root / split / "labels" / f"{json_file.stem}.txt"
    with open(label_path, "w") as f:
        f.write("\n".join(yolo_labels))

    # Save/copy image
    cv2.imwrite(str(image_path), image)

# Process all JSONs
for split in splits:
    ann_dir = dataset_root / "annotations" / split
    for json_file in ann_dir.glob("*.json"):
        convert_to_yolo_bbox(json_file, split)

# Write YOLOv8 data.yaml
yaml_path = dataset_root / "data.yaml"
with open(yaml_path, "w") as f:
    f.write(f"""path: {dataset_root}
train: train/images
val: val/images
names: {all_labels}
""")

print("✅ YOLOv8 detection label conversion complete.")

✅ YOLOv8 detection label conversion complete.


## YOLOv8 Seg Model Training

In [10]:
# Path to the generated dataset and YAML file
dataset_path = Path("dataset")
data_yaml = dataset_path / "data.yaml"

# Load a YOLOv8 **detection** model (not segmentation!)
model = YOLO("yolov8s.pt")  # ✅ remove '-seg' from the model name

# Train the model
model.train(
    data=str(data_yaml),
    epochs=300,
    imgsz=512,
    batch=8,
    project="yolo-det-defect",
    name="yolov8-det-train"
)

100%|██████████| 21.5M/21.5M [00:06<00:00, 3.54MB/s]


New https://pypi.org/project/ultralytics/8.3.143 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.140  Python-3.10.11 torch-2.7.0+cpu CPU (Intel Core(TM) i9-7900X 3.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8-det-train, nbs=64, nms=False, opset=None, optimize

train: Scanning C:\Users\uos\Desktop\FEMM Hub\Project 1 - Multimodal Inspection Rig\Project Code\multimodal-inspection-rig\YOLO_defectDetect\dataset\train\labels.cache... 96 images, 0 backgrounds, 0 corrupt: 100%|██████████| 96/96 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 138.4108.4 MB/s, size: 102.3 KB)



c:\Users\uos\Desktop\FEMM Hub\Project 1 - Multimodal Inspection Rig\Project Code\multimodal-inspection-rig\YOLO_defectDetect\.venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\uos\Desktop\FEMM Hub\Project 1 - Multimodal Inspection Rig\Project Code\multimodal-inspection-rig\YOLO_defectDetect\dataset\val\labels.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]

...

                   all         24         94      0.611      0.667       0.62      0.287
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 144, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



244 epochs completed in 1.703 hours.
Optimizer stripped from yolo-det-defect\yolov8-det-train\weights\last.pt, 22.5MB
Optimizer stripped from yolo-det-defect\yolov8-det-train\weights\best.pt, 22.5MB

Validating yolo-det-defect\yolov8-det-train\weights\best.pt...
Ultralytics 8.3.140  Python-3.10.11 torch-2.7.0+cpu CPU (Intel Core(TM) i9-7900X 3.30GHz)
Model summary (fused): 72 layers, 11,127,519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


                   all         24         94      0.731      0.589      0.668      0.322
                cavity          5          5       0.57        0.6      0.685      0.386
            coil_break         10         10      0.584       0.15      0.293     0.0559
          coil_overlap         14         17      0.788      0.765      0.727       0.37
              fracture          8         11      0.762      0.636      0.712      0.264
                  hole         19         51      0.953      0.796       0.92      0.533
Speed: 1.0ms preprocess, 63.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to yolo-det-defect\yolov8-det-train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000176BFD4D0C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
    

## Real Time Part Detection using Intel D435i and YOLOv8

In [46]:
YOLO_DETECT_VID_PATH = "capturedVideos/"

In [50]:
import cv2
import pyrealsense2 as rs
import numpy as np
from ultralytics import YOLO
from datetime import datetime

# Load YOLOv8 model
model = YOLO("yolo_det_project/yolov8-det-train/weights/best.pt")  # replace with your model path

# RealSense camera setup
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)

# Video recording setup
recording = False
video_writer = None
video_filename = None
fps = 30
frame_size = (640, 480)

print("Press SPACE to start/stop recording, ESC to exit.")

try:
    while True:
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        frame = np.asanyarray(color_frame.get_data())

        # Run YOLOv8 inference
        results = model.predict(source=frame, save=False, stream=False, show=False, verbose=False)
        annotated = results[0].plot()

        # Show frame
        cv2.imshow("YOLOv8 RealSense Inference", annotated)

        # Write to video if recording
        if recording and video_writer:
            video_writer.write(annotated)

        key = cv2.waitKey(1) & 0xFF

        # SPACE: Toggle recording
        if key == 32:
            recording = not recording
            if recording:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                video_filename = YOLO_DETECT_VID_PATH + f"output_{timestamp}.avi"
                video_writer = cv2.VideoWriter(
                    video_filename,
                    cv2.VideoWriter_fourcc(*'XVID'),
                    fps,
                    frame_size
                )
                print(f"Recording started: {video_filename}")
            else:
                if video_writer:
                    video_writer.release()
                    print(f"Recording saved: {video_filename}")
                    video_writer = None

        # ESC: Exit
        elif key == 27:
            print("ESC pressed. Exiting...")
            break

finally:
    pipeline.stop()
    if video_writer:
        video_writer.release()
    cv2.destroyAllWindows()
    print("Cleanup complete.")

Press SPACE to start/stop recording, ESC to exit.
Recording started: capturedVideos/output_20250523_111623.avi
Recording saved: capturedVideos/output_20250523_111623.avi
ESC pressed. Exiting...
Cleanup complete.
